# MCO 1 - 2012 Family Income and Expenditure Survey (FIES)
In this Notebook, we will explore income and expenditure behavior across Filipino households using the 2012 Family Income and Expenditure Survey (FIES) dataset. We will focus on statistical inference, particularly confidence intervals and hypothesis testing for means, while also applying unsupervised learning techniques such as clustering to reveal patterns in household spending.

We aim to understand how households from different income groups allocate their spending across essential categories like food, education, and utilities.

The dataset, provided in the file FIES PUF 2012 Vol.1.CSV, comes from the Philippine Statistics Authority and contains anonymized microdata on household income from various sources (such as salaries, businesses, and remittances), categorized expenditures (including food, housing, education, health, and utilities), as well as demographic and geographic variables like region and urban/rural classification. Household characteristics such as household size and number of earners are also included. 

## Research Questions

### General Research Question:
What are the key differences in expenditure allocation (e.g., food, education, utilities) across income groups?

#### Supporting Research Questions:
1. What are the average and median incomes in each income group?
2. Which expenditure category takes up the largest portion of total expenses for each group?
3. Do wealthier households spend a higher or lower percentage of their income on basic needs like food and utilities?
4. Are low-income households more likely to prioritize essential expenses over discretionary (e.g., entertainment, travel) ones?
5. How does the ratio of entertainment spending to income change as income increases?
6. How does the ratio of education spending to income change as income increases?
7. Is there a statistically significant difference in food expenditure between the lowest and highest income groups?



## Importing Libraries

We will import essential Python libraries for data analysis and visualization. `numpy` is used for numerical operations, `pandas` for data manipulation and analysis, `matplotlib.pyplot` for creating visualizations, and `scipy.stats` for statistical functions and hypothesis testing.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import ttest_ind

## Family Income and Expenditure Data


We will load the Family Income and Expenditure Survey (FIES) dataset from the provided CSV file using pandas' `read_csv` function. After loading, we will use the `head()` method to display the first few rows and get an initial look at the data structure and contents.

In [2]:
fies_df = pd.read_csv('./Dataset/FIES_PUF_2012_Vol.1.CSV')
fies_df.head()

,W_REGN,W_OID,W_SHSN,W_HCN,URB,RSTR,PSU,BWEIGHT,RFACT,FSIZE,...,PC_QTY,OVEN_QTY,MOTOR_BANCA_QTY,MOTORCYCLE_QTY,POP_ADJ,PCINC,NATPC,NATDC,REGDC,REGPC
0,14,101001000,2,25,2,21100,415052,138.25,200.6576,3.0,...,1,1,,,0.946172,108417.00,9,8,8,9
1,14,101001000,3,43,2,21100,415052,138.25,200.6576,12.5,...,,1,,1,0.946172,30631.60,5,9,9,4
2,14,101001000,4,62,2,21100,415052,138.25,200.6576,2.0,...,,1,,,0.946172,86992.50,9,6,6,8
3,14,101001000,5,79,2,21100,415052,138.25,200.6576,4.0,...,,1,,,0.946172,43325.75,6,6,6,6
4,14,101001000,10,165,2,21100,415052,138.25,200.6576,5.0,...,,,,1,0.946172,37481.80,6,6,6,5


We will use the `info()` function to get a concise summary of the DataFrame, including the number of non-null entries, column data types, and memory usage. This helps us understand the structure and completeness of the dataset.

In [3]:
fies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40171 entries, 0 to 40170
Columns: 119 entries, W_REGN to REGPC
dtypes: float64(5), int64(92), object(22)
memory usage: 36.5+ MB


Next, we will use the `describe()` function to generate descriptive statistics for the numerical columns, such as mean, standard deviation, minimum, and maximum values. This provides an overview of the distribution and central tendencies of the data.

In [4]:
fies_df.describe()

,W_REGN,W_OID,W_SHSN,W_HCN,URB,RSTR,PSU,BWEIGHT,RFACT,FSIZE,...,HSE_ALTERTN,TOILET,ELECTRIC,WATER,POP_ADJ,PCINC,NATPC,NATDC,REGDC,REGPC
count,40171.000000,4.017100e+04,40171.000000,40171.000000,40171.000000,40171.000000,40171.000000,40171.000000,40171.000000,40171.000000,...,40171.000000,40171.000000,40171.000000,40171.000000,40171.000000,4.017100e+04,40171.000000,40171.000000,40171.000000,40171.000000
mean,13.019890,4.210536e+09,9.633666,1563.601753,1.617311,21547.277215,258123.702099,340.330363,533.363298,4.699223,...,1.940330,1.718130,1.131563,3.186030,0.942329,5.432433e+04,5.233303,5.238306,5.445769,5.455129
std,11.995555,2.285729e+09,6.198442,2977.363506,0.486049,3520.981146,112143.268816,112.377931,209.996517,2.194050,...,0.236877,1.539145,0.338019,2.405758,0.038631,7.372111e+04,2.874581,2.856486,2.866703,2.864137
min,1.000000,1.010010e+08,1.000000,1.000000,1.000000,2475.000000,100010.000000,92.250000,126.164300,1.000000,...,1.000000,0.000000,1.000000,1.000000,0.876132,2.979200e+03,1.000000,1.000000,1.000000,1.000000
25%,6.000000,2.239012e+09,4.000000,95.000000,1.000000,21100.000000,116384.000000,271.500000,399.615000,3.000000,...,2.000000,1.000000,1.000000,1.000000,0.924450,1.996803e+04,3.000000,3.000000,3.000000,3.000000
50%,10.000000,4.112005e+09,9.000000,204.000000,2.000000,22100.000000,216212.000000,329.750000,509.874900,4.500000,...,2.000000,1.000000,1.000000,3.000000,0.940724,3.336975e+04,5.000000,5.000000,5.000000,5.000000
75%,14.000000,6.210006e+09,14.000000,393.000000,2.000000,23200.000000,316519.000000,428.710000,634.160800,6.000000,...,2.000000,2.000000,1.000000,4.000000,0.961401,6.175867e+04,8.000000,8.000000,8.000000,8.000000
max,42.000000,9.804035e+09,30.000000,8026.000000,2.000000,29000.000000,416581.000000,1630.200000,2895.814900,20.500000,...,2.000000,7.000000,2.000000,12.000000,1.058416,3.231120e+06,10.000000,10.000000,10.000000,10.000000


### Data Cleaning

#### Multiple Representations in Data Cleaning

When cleaning data, it is important to check for multiple representations of the same value, especially in categorical columns. Multiple representations can occur when the same category is recorded in different ways (e.g., "Male", "male", "M", or "m"). This can lead to inaccurate analysis and misleading results. In this dataset, we examined the categorical columns to identify any such inconsistencies. 

In [5]:
# Check if there are categorical columns
categorical_columns = fies_df.select_dtypes(include=['object']).columns
print("Categorical columns:", categorical_columns)

Categorical columns: Index(['OCCUP', 'KB', 'CW', 'AGELESS5', 'AGE5_17', 'EMPLOYED_PAY',
       'EMPLOYED_PROF', 'DISTANCE', 'RADIO_QTY', 'TV_QTY', 'CD_QTY',
       'STEREO_QTY', 'REF_QTY', 'WASH_QTY', 'AIRCON_QTY', 'CAR_QTY',
       'LANDLINE_QTY', 'CELLPHONE_QTY', 'PC_QTY', 'OVEN_QTY',
       'MOTOR_BANCA_QTY', 'MOTORCYCLE_QTY'],
      dtype='object')


There are multiple categorical columns, but upon manual inspection, these were flagged as categorical due to having spaces as values instead of NaN. We should replace continuous spaces with np.nan, convert these columns into numeric types if applicable, then print categorical columns again to verify.

In [6]:
# Replace cells with only spaces or empty strings with np.nan
fies_df = fies_df.replace(r'^\s*$', np.nan, regex=True)

# Convert columns that can be numeric to numeric types
for col in fies_df.columns:
    fies_df[col] = pd.to_numeric(fies_df[col], errors='ignore')

# Now get categorical columns (should not include numeric columns with NaN)
categorical_columns = fies_df.select_dtypes(include=['object']).columns
print("Categorical columns:", categorical_columns)

Categorical columns: Index([], dtype='object')


C:\Users\Diane\AppData\Local\Temp\ipykernel_2540\2939810117.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  fies_df[col] = pd.to_numeric(fies_df[col], errors='ignore')


Since there are no categorical columns present, there are no issues with multiple representations in this case. Moreover, there are also no cases of inconsistent formatting in the dataset.

#### Checking for Incorrect Data Types

It is important to ensure that each column in the dataset contains consistent data types. Inconsistent data types within a column (e.g., mixing strings and numbers) can lead to errors during analysis and may affect the results of computations or visualizations. To check for inconsistent data types, we examine each column to see if it contains more than one type of data. If any columns are found with mixed types, they may require cleaning or conversion to a uniform type before proceeding with further analysis.

In [7]:
# Check if any column contains more than one data type
numOfColumns = 0;

for col in fies_df.columns:
    types_in_col = fies_df[col].apply(type).unique()
    if len(types_in_col) > 1:
        numOfColumns += 1
print("Number of columns with mixed data types:", numOfColumns)

Number of columns with mixed data types: 0


After checking each column for mixed data types, we found that all columns in the dataset contain consistent data types. This ensures that the data is clean and ready for further analysis without the need for additional type conversions.

#### Treating Missing Values

Handling missing values is an essential part of data preprocessing to ensure the accuracy and reliability of our analysis. In this dataset, missing values are primarily found in columns related to household item quantities, which were collected only during the second visit. Instead of removing these columns or focusing solely on the first visit, we will retain all available data and acknowledge the presence of missing values in our analysis.

For columns with missing values, we will use appropriate strategies such as imputation or analysis with available data, depending on the context and research question. This approach allows us to maximize the use of the dataset while maintaining transparency about data limitations.

In [20]:
# Show only columns with missing values and their missing percentage
missing_cols = fies_df.columns[fies_df.isnull().any()]
missing_percent = fies_df[missing_cols].isnull().mean() * 100

print("Columns with missing values and their percentages:")
print(missing_percent.round(2).astype(str) + '%')

Columns with missing values and their percentages:
OCCUP              18.36%
KB                 18.36%
CW                 18.36%
AGELESS5           38.24%
AGE5_17             20.5%
EMPLOYED_PAY       20.65%
EMPLOYED_PROF      31.61%
DISTANCE           41.84%
RADIO_QTY          39.96%
TV_QTY             19.45%
CD_QTY             35.51%
STEREO_QTY         57.29%
REF_QTY            44.26%
WASH_QTY           50.05%
AIRCON_QTY          63.8%
CAR_QTY            65.11%
LANDLINE_QTY       65.05%
CELLPHONE_QTY      15.99%
PC_QTY             58.33%
OVEN_QTY           60.08%
MOTOR_BANCA_QTY    68.55%
MOTORCYCLE_QTY     57.57%
dtype: object


Although there are many missing values in the dataset, they are primarily found in columns that are not essential for answering our research questions. Therefore, we can leave these missing values as is for now, especially since removing them would result in a significant loss of data.

#### Treating duplicates

To ensure data integrity and avoid skewed analysis, it is important to check for and remove duplicate rows in the dataset. Duplicate entries can occur due to data entry errors or merging datasets from multiple sources. By using the `drop_duplicates()` function, we can identify and remove any repeated rows. We also compare the number of rows before and after this operation to confirm how many duplicates, if any, were found and removed.

In [9]:
# Drop duplicate rows based on all columns

initial_rows = fies_df.shape[0]
fies_df.drop_duplicates(inplace=True)
final_rows = fies_df.shape[0]
print(f"Initial categories: {initial_rows}, Final categories after dropping duplicates: {final_rows}")


Initial categories: 40171, Final categories after dropping duplicates: 40171


Here, we can see that after applying the drop_duplicate function, the number of categories remains the same, which means that no categories have been duplicated.


#### Treating Outliers

- to be added

Start here for Q1

Start here for Q2

Start here for Q3

Start here for Q4

Start here for Q5

Start here for Q6

Start here for Q7